### Prefix Sums

In [1]:
x = [1, 4, 3, 5, 6, 7, 0, 1]

rdd = sc.parallelize(x, 4).cache()

def f(iterator):
    yield sum(iterator)

sums = rdd.mapPartitions(f).collect()

print(sums)

for i in range(1, len(sums)):
    sums[i] += sums[i-1]

print(sums)

def g(index, iterator):
    global sums
    if index == 0:
        s = 0
    else:
        s = sums[index-1]
    for i in iterator:
        s += i
        yield s

prefix_sums = rdd.mapPartitionsWithIndex(g)
print(prefix_sums.collect())

[5, 8, 13, 1]
[5, 13, 26, 27]
[1, 5, 8, 13, 19, 26, 26, 27]


### Monotocity checking

In [2]:
x = [1, 3, 4, 5, 7, 3, 10, 14, 16, 20, 21, 24, 24, 26, 27, 30]

rdd = sc.parallelize(x, 4).cache()

def f(it):
    first = next(it) #return first item in the generator
    last = first
    increasing = True
    for i in it:
        if i < last: #not montically increasing
            increasing = False
        last = i
    yield increasing, first, last #always get 

results = rdd.mapPartitions(f).collect()

print(results)

increasing = True
if results[0][0] == False:
    increasing = False
else:
    for i in range(1, len(results)):
        if results[i][0] == False or results[i][1] < results[i-1][2]:
            increasing = False
if increasing:
    print("Monotone")
else:
    print("Not monotone")


[(True, 1, 5), (False, 7, 14), (True, 16, 24), (True, 24, 30)]
Not monotone


### Maximum Subarray Problem

In [3]:
# Classical divide and conquer algorithm

A = [-3, 2, 1, -4, 5, 2, -1, 3, -1]

def MaxSubarray(A, p, r):
    if p == r:
        return A[p]
    q = (p+r)//2
    M1 = MaxSubarray(A, p, q)
    M2 = MaxSubarray(A, q+1, r)
    Lm = -float('inf')
    Rm = Lm
    V = 0
    for i in range(q, p-1, -1):
        V += A[i]
        if V > Lm:
            Lm = V
    V = 0
    for i in range(q+1, r+1):
        V += A[i]
        if V > Rm:
            Rm = V
    return max(M1, M2, Lm+Rm)

print(MaxSubarray(A, 0, len(A)-1))

9


In [4]:
# Linear-time algorithm
# Written in a way so that we can call it for each partition

def linear_time(it):
    Vmax = -float('inf')
    V = 0
    for Ai in it:
        V += Ai
        if V > Vmax:
            Vmax = V
        if V < 0:
            V = 0
    yield Vmax
    
print(next(linear_time(A)))

9


In [5]:
# The Spark algorithm:

def compute_sum(it):
    yield sum(it)

def compute_LmRm(index, it):
    Rm = -float('inf')
    L = sums[index]
    Lm = L
    R = 0
    for Ai in it:
        L -= Ai
        R += Ai
        if L > Lm:
            Lm = L
        if R > Rm:
            Rm = R
    yield (Lm, Rm)

num_partitions = 4
rdd = sc.parallelize(A, num_partitions).cache()
print(rdd.collect())
sums = rdd.mapPartitions(compute_sum).collect()
print(sums)
LmRms = rdd.mapPartitionsWithIndex(compute_LmRm).collect()
print(LmRms)
best = max(rdd.mapPartitions(linear_time).collect())

for i in range(num_partitions-1):
    for j in range(i+1, num_partitions):
        x = LmRms[i][0] + sum(sums[i+1:j]) + LmRms[j][1]
        if x > best:
            best = x

print(best)

[-3, 2, 1, -4, 5, 2, -1, 3, -1]
[-1, -3, 7, 1]
[(2, -1), (0, 1), (7, 7), (2, 2)]
9


In [14]:
import time
from operator import add

data = sc.parallelize(range(0,10)).filter(lambda x: x<=int(time.time())%10)

data.map(lambda x:x+1).reduce(add) + data.map(lambda x: x-1).reduce(add),data.reduce(add)*2

(42, 56)

0